In [1]:
import sys
sys.path.insert(0, '/nfs/general/shared/code/rpi_color_detector/classifier/')
import image_provider
import tensorflow as tf
import keras_tuner as kt
import tensorflow_model_optimization as tfmot


In [4]:
def TrainingDataGen():
    ip = image_provider.ImageProvider()
    d = ip.listImages('classified')
    validClassNames = sorted(
        [x for x in d if x not in ['unclassified', '_garbage']])
    #tf.float32()
    for i, className in enumerate(validClassNames):
        for image_path in d[className]:
            yield ip.filePath(image_path), i


def LoadImage(filename):
    image = tf.io.read_file(filename)
    image = tf.io.decode_jpeg(image)
    image.set_shape([480, 640, 3])
    return image

data_augmentation = tf.keras.Sequential(
    [
        tf.keras.layers.RandomTranslation(0.05, 0.05)
    ]
)

AUTOTUNE = tf.data.AUTOTUNE

all_data = tf.data.Dataset.from_generator(TrainingDataGen, output_signature=(
    tf.TensorSpec(shape=(), dtype=tf.string), tf.TensorSpec(shape=(), dtype=tf.int32)))
all_data = all_data.map(lambda imgName, label : (LoadImage(imgName), label), deterministic=False, num_parallel_calls=AUTOTUNE)
all_data = all_data.shuffle(buffer_size=10000)

training_data = all_data.skip(400).cache().repeat()
training_data = training_data.map(lambda x, y: (data_augmentation(x), y), num_parallel_calls=AUTOTUNE, deterministic=False)
training_data = training_data.batch(64)
training_data = training_data.prefetch(buffer_size=AUTOTUNE)

validation_data = all_data.take(400).cache()
validation_data = validation_data.map(lambda x, y: (data_augmentation(x), y), num_parallel_calls=AUTOTUNE, deterministic=False)
validation_data = validation_data.batch(64)
validation_data = validation_data.prefetch(buffer_size=AUTOTUNE)


class ExtractRoiForPrediction(tf.keras.layers.Layer):
    def __init__(self, crop_factor, **kwargs):
        super(ExtractRoiForPrediction, self).__init__()
        self.crop_factor = crop_factor

    def get_config(self):
        config = super().get_config()
        config.update({
            "crop_factor": self.crop_factor,
        })
        return config

    def call(self, inputs):
        x=self.crop_factor
        target_width=int(640*x)
        target_height=int(480*x)
        offset_width=(640-target_width)//2
        offset_height=(480-target_height)//2
        img = tf.image.crop_to_bounding_box(inputs, offset_height, offset_width, target_height, target_width)
        return tf.cast(img, tf.float32)/255.0


class DoNotQuantizeConfig(tfmot.quantization.keras.QuantizeConfig):
    def get_weights_and_quantizers(self, layer):
      return []
    def get_activations_and_quantizers(self, layer):
      return []
    def set_quantize_weights(self, layer, quantize_weights):
      pass
    def set_quantize_activations(self, layer, quantize_activations):
      pass
    def get_output_quantizers(self, layer):
      return []
    def get_config(self):
      return {}

model =  tfmot.quantization.keras.quantize_annotate_model(tf.keras.Sequential([
    tfmot.quantization.keras.quantize_annotate_layer(ExtractRoiForPrediction(0.3), DoNotQuantizeConfig()),
    tf.keras.layers.Conv2D(8, (5, 5), padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(8, (3, 3), padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(8, (3, 3), padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(7)
]))

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=5000,
    decay_rate=0.9,
    staircase=True)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),  # 'adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(
                  from_logits=True),
              metrics=['accuracy'])

tb_callback = tf.keras.callbacks.TensorBoard(
    '/tmp/logs', update_freq=1, histogram_freq=1, write_images=True)

checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

model.fit(training_data, epochs=1000, steps_per_epoch=32, validation_data=validation_data, callbacks=[tb_callback, model_checkpoint_callback], verbose=2)

tf.saved_model.save(model, '/nfs/general/shared/tf_models/color_classifier')

with  tfmot.quantization.keras.quantize_scope(
  {'DoNotQuantizeConfig': DoNotQuantizeConfig,
   'ExtractRoiForPrediction': ExtractRoiForPrediction}):
  q_aware_model = tfmot.quantization.keras.quantize_apply(model)

q_aware_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),  # 'adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(
                  from_logits=True),
              metrics=['accuracy'])

q_aware_model.fit(training_data, epochs=400, steps_per_epoch=32, validation_data=validation_data, callbacks=[tb_callback, model_checkpoint_callback], verbose=2)

tf.saved_model.save(model, '/nfs/general/shared/tf_models/color_classifier_quant_aware')

converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model) # ('/nfs/general/shared/tf_models/color_classifier_quant_aware') # path to the SavedModel directory
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
open('/nfs/general/shared/tflite/color_classifier.tflite', 'wb').write(tflite_model)

Epoch 1/1000
32/32 - 4s - loss: 1.9331 - accuracy: 0.1797 - val_loss: 1.8532 - val_accuracy: 0.2550 - 4s/epoch - 140ms/step
Epoch 2/1000
32/32 - 3s - loss: 1.6463 - accuracy: 0.3784 - val_loss: 1.4127 - val_accuracy: 0.6100 - 3s/epoch - 95ms/step
Epoch 3/1000
32/32 - 3s - loss: 1.4073 - accuracy: 0.5181 - val_loss: 1.2396 - val_accuracy: 0.5575 - 3s/epoch - 84ms/step
Epoch 4/1000
32/32 - 4s - loss: 1.1876 - accuracy: 0.5410 - val_loss: 1.0097 - val_accuracy: 0.6925 - 4s/epoch - 139ms/step
Epoch 5/1000
32/32 - 2s - loss: 1.0641 - accuracy: 0.5610 - val_loss: 0.7750 - val_accuracy: 0.7025 - 2s/epoch - 76ms/step
Epoch 6/1000
32/32 - 2s - loss: 0.8758 - accuracy: 0.5830 - val_loss: 0.6426 - val_accuracy: 0.7125 - 2s/epoch - 76ms/step
Epoch 7/1000
32/32 - 2s - loss: 0.8292 - accuracy: 0.5825 - val_loss: 0.5295 - val_accuracy: 0.7000 - 2s/epoch - 76ms/step
Epoch 8/1000
32/32 - 2s - loss: 0.7596 - accuracy: 0.5869 - val_loss: 0.5574 - val_accuracy: 0.7000 - 2s/epoch - 76ms/step
Epoch 9/1000
3

2022-04-24 14:15:36.747444: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /nfs/general/shared/tf_models/color_classifier/assets


INFO:tensorflow:Assets written to: /nfs/general/shared/tf_models/color_classifier/assets


Epoch 1/400
32/32 - 3s - loss: 0.4016 - accuracy: 0.8687 - val_loss: 0.1332 - val_accuracy: 0.9900 - 3s/epoch - 107ms/step
Epoch 2/400
32/32 - 3s - loss: 0.3262 - accuracy: 0.8384 - val_loss: 0.1734 - val_accuracy: 0.8675 - 3s/epoch - 83ms/step
Epoch 3/400
32/32 - 3s - loss: 0.3318 - accuracy: 0.8081 - val_loss: 0.1549 - val_accuracy: 0.8725 - 3s/epoch - 86ms/step
Epoch 4/400
32/32 - 3s - loss: 0.3271 - accuracy: 0.7969 - val_loss: 0.1630 - val_accuracy: 0.8725 - 3s/epoch - 79ms/step
Epoch 5/400
32/32 - 3s - loss: 0.2931 - accuracy: 0.8242 - val_loss: 0.1549 - val_accuracy: 0.8775 - 3s/epoch - 81ms/step
Epoch 6/400
32/32 - 3s - loss: 0.2988 - accuracy: 0.8359 - val_loss: 0.0850 - val_accuracy: 0.9725 - 3s/epoch - 79ms/step
Epoch 7/400
32/32 - 3s - loss: 0.1937 - accuracy: 0.9224 - val_loss: 0.0491 - val_accuracy: 0.9950 - 3s/epoch - 78ms/step
Epoch 8/400
32/32 - 2s - loss: 0.2007 - accuracy: 0.9258 - val_loss: 0.0215 - val_accuracy: 0.9950 - 2s/epoch - 78ms/step
Epoch 9/400
32/32 - 2s 

INFO:tensorflow:Assets written to: /nfs/general/shared/tf_models/color_classifier_quant_aware/assets


INFO:tensorflow:Assets written to: /nfs/general/shared/tf_models/color_classifier_quant_aware/assets


INFO:tensorflow:Assets written to: /tmp/tmpkpelkumz/assets


INFO:tensorflow:Assets written to: /tmp/tmpkpelkumz/assets
/home/geli/.local/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2022-04-24 14:32:28.732839: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2022-04-24 14:32:28.732867: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2022-04-24 14:32:28.733468: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpkpelkumz
2022-04-24 14:32:28.737973: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2022-04-24 14:32:28.737985: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /tmp/tmpkpelkumz
2022-04-24 14:32:28.752087: I tensorflow/cc/saved_model/loader.cc:22

64872

In [6]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)


with  tfmot.quantization.keras.quantize_scope(
  {'DoNotQuantizeConfig': DoNotQuantizeConfig,
   'ExtractRoiForPrediction': ExtractRoiForPrediction}):
  q_aware_model = tfmot.quantization.keras.quantize_apply(model)

q_aware_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(
                  from_logits=True),
              metrics=['accuracy'])

q_aware_model.fit(training_data, epochs=50, steps_per_epoch=32, validation_data=validation_data, callbacks=[tb_callback, model_checkpoint_callback], verbose=2)

tf.saved_model.save(model, '/nfs/general/shared/tf_models/color_classifier_quant_aware')

converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model) # ('/nfs/general/shared/tf_models/color_classifier_quant_aware') # path to the SavedModel directory
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
open('/nfs/general/shared/tflite/color_classifier.tflite', 'wb').write(tflite_model)

Epoch 1/50
32/32 - 3s - loss: 0.4181 - accuracy: 0.8687 - val_loss: 0.1570 - val_accuracy: 0.9925 - 3s/epoch - 104ms/step
Epoch 2/50
32/32 - 3s - loss: 0.3158 - accuracy: 0.8364 - val_loss: 0.1747 - val_accuracy: 0.8700 - 3s/epoch - 83ms/step
Epoch 3/50
32/32 - 3s - loss: 0.3124 - accuracy: 0.8198 - val_loss: 0.1583 - val_accuracy: 0.8675 - 3s/epoch - 79ms/step
Epoch 4/50
32/32 - 3s - loss: 0.3248 - accuracy: 0.8071 - val_loss: 0.1604 - val_accuracy: 0.8725 - 3s/epoch - 85ms/step
Epoch 5/50
32/32 - 3s - loss: 0.2802 - accuracy: 0.8247 - val_loss: 0.1390 - val_accuracy: 0.8975 - 3s/epoch - 78ms/step
Epoch 6/50
32/32 - 3s - loss: 0.2390 - accuracy: 0.9048 - val_loss: 0.0313 - val_accuracy: 0.9875 - 3s/epoch - 79ms/step
Epoch 7/50
32/32 - 3s - loss: 0.1954 - accuracy: 0.9082 - val_loss: 0.0215 - val_accuracy: 0.9975 - 3s/epoch - 79ms/step
Epoch 8/50
32/32 - 3s - loss: 0.1654 - accuracy: 0.9453 - val_loss: 0.0310 - val_accuracy: 0.9950 - 3s/epoch - 79ms/step
Epoch 9/50
32/32 - 2s - loss: 0

INFO:tensorflow:Assets written to: /nfs/general/shared/tf_models/color_classifier_quant_aware/assets


INFO:tensorflow:Assets written to: /nfs/general/shared/tf_models/color_classifier_quant_aware/assets


INFO:tensorflow:Assets written to: /tmp/tmpy6m_9cg6/assets


INFO:tensorflow:Assets written to: /tmp/tmpy6m_9cg6/assets
/home/geli/.local/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2022-04-24 15:01:57.144584: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2022-04-24 15:01:57.144616: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2022-04-24 15:01:57.144810: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpy6m_9cg6
2022-04-24 15:01:57.149186: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2022-04-24 15:01:57.149201: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /tmp/tmpy6m_9cg6
2022-04-24 15:01:57.160403: I tensorflow/cc/saved_model/loader.cc:22

64880

In [7]:
converter = tf.lite.TFLiteConverter.from_keras_model(model) # ('/nfs/general/shared/tf_models/color_classifier_quant_aware') # path to the SavedModel directory
tflite_model = converter.convert()
open('/nfs/general/shared/tflite/color_classifier_non_quatized.tflite', 'wb').write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpnqfy59vl/assets


INFO:tensorflow:Assets written to: /tmp/tmpnqfy59vl/assets
2022-04-24 15:02:47.897943: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2022-04-24 15:02:47.897968: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2022-04-24 15:02:47.898174: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpnqfy59vl
2022-04-24 15:02:47.900974: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2022-04-24 15:02:47.900985: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /tmp/tmpnqfy59vl
2022-04-24 15:02:47.907405: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-04-24 15:02:47.949735: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: /tmp/tmpnqfy59vl
2022-04-24 15:02:47.964127: I tensorflow/cc/saved_model/loader.cc:301] SavedModel

234548

In [ ]:
def TrainingDataGen():
    ip = image_provider.ImageProvider()
    d = ip.listImages('classified')
    validClassNames = sorted(
        [x for x in d if x not in ['unclassified', '_garbage']])
    for i, className in enumerate(validClassNames):
        for image_path in d[className]:
            yield ip.filePath(image_path), i


def LoadImage(filename):
    image = tf.io.read_file(filename)
    image = tf.io.decode_jpeg(image)
    image.set_shape([480, 640, 3])
    return tf.image.convert_image_dtype(image, tf.float32)

data_augmentation = tf.keras.Sequential(
    [
        tf.keras.layers.RandomTranslation(0.05, 0.05)
    ]
)

AUTOTUNE = tf.data.AUTOTUNE

all_data = tf.data.Dataset.from_generator(TrainingDataGen, output_signature=(
    tf.TensorSpec(shape=(), dtype=tf.string), tf.TensorSpec(shape=(), dtype=tf.int32)))
all_data = all_data.map(lambda imgName, label : (LoadImage(imgName), label), deterministic=False, num_parallel_calls=AUTOTUNE)
all_data = all_data.shuffle(buffer_size=10000)

training_data = all_data.skip(400).cache().repeat()
training_data = training_data.map(lambda x, y: (data_augmentation(x), y), num_parallel_calls=AUTOTUNE, deterministic=False)
training_data = training_data.batch(64)
training_data = training_data.prefetch(buffer_size=AUTOTUNE)

validation_data = all_data.take(400).cache()
validation_data = validation_data.map(lambda x, y: (data_augmentation(x), y), num_parallel_calls=AUTOTUNE, deterministic=False)
validation_data = validation_data.batch(64)
validation_data = validation_data.prefetch(buffer_size=AUTOTUNE)


class ExtractRoiForPrediction(tf.keras.layers.Layer):
    def __init__(self, crop_factor, **kwargs):
        super(ExtractRoiForPrediction, self).__init__()
        self.crop_factor = crop_factor

    def get_config(self):
        config = super().get_config()
        config.update({
            "crop_factor": self.crop_factor,
        })
        return config

    def call(self, inputs):
        x=self.crop_factor
        target_width=int(640*x)
        target_height=int(480*x)
        offset_width=(640-target_width)//2
        offset_height=(480-target_height)//2
        return tf.image.crop_to_bounding_box(inputs, offset_height, offset_width, target_height, target_width)

model =  tfmot.quantization.keras.quantize_annotate_model(tf.keras.Sequential([
    tfmot.quantization.keras.quantize_annotate_layer(ExtractRoiForPrediction(0.3), DoNotQuantizeConfig()),
    tf.keras.layers.Conv2D(8, (5, 5), padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(8, (3, 3), padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(8, (3, 3), padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(7)
]))

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=5000,
    decay_rate=0.9,
    staircase=True)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),  # 'adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(
                  from_logits=True),
              metrics=['accuracy'])

tb_callback = tf.keras.callbacks.TensorBoard(
    '/tmp/logs', update_freq=1, histogram_freq=1, write_images=True)

checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=False)

model.fit(training_data, epochs=1000, steps_per_epoch=32, validation_data=validation_data, callbacks=[tb_callback, model_checkpoint_callback], verbose=2)

tf.saved_model.save(model, '/nfs/general/shared/tf_models/color_classifier')

with  tfmot.quantization.keras.quantize_scope(
  {'DoNotQuantizeConfig': DoNotQuantizeConfig,
   'ExtractRoiForPrediction': ExtractRoiForPrediction}):
  q_aware_model = tfmot.quantization.keras.quantize_apply(model)

q_aware_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),  # 'adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(
                  from_logits=True),
              metrics=['accuracy'])

q_aware_model.fit(training_data, epochs=1200, steps_per_epoch=32, validation_data=validation_data, callbacks=[tb_callback, model_checkpoint_callback], verbose=2)

tf.saved_model.save(model, '/nfs/general/shared/tf_models/color_classifier_quant_aware')

converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model) # ('/nfs/general/shared/tf_models/color_classifier_quant_aware') # path to the SavedModel directory
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
open('/nfs/general/shared/tflite/color_classifier.tflite', 'wb').write(tflite_model)

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model('/nfs/general/shared/tf_models/color_classifier3') # path to the SavedModel directory
tflite_model = converter.convert()
open('/nfs/general/shared/tflite/color_classifier.tflite', 'wb').write(tflite_model)

In [ ]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.0005,
    decay_steps=1000,
    decay_rate=0.9,
    staircase=True)

q_aware_model.fit(training_data, epochs=1200, steps_per_epoch=32, validation_data=validation_data, callbacks=[tb_callback, model_checkpoint_callback], verbose=2)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model) # ('/nfs/general/shared/tf_models/color_classifier_quant_aware') # path to the SavedModel directory
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
open('/nfs/general/shared/tflite/color_classifier.tflite', 'wb').write(tflite_model)